<a href="https://colab.research.google.com/github/farrukh61/Physics-informed-neural-networks/blob/main/PINNS_Mazair_Burgers_Equation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##https://github.com/maziarraissi/PINNs/blob/master/appendix/continuous_time_identification%20(Burgers)/Burgers.py


from google.colab import drive

# drive.mount('/drive')
import sys
import tensorflow as tf
import numpy as np
import scipy.io
import time
from scipy.interpolate import griddata

from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [2]:
drive.mount('/drive')

Mounted at /drive


In [11]:
np.random.seed(1234)
tf.random.set_seed(1234)

mat_data = scipy.io.loadmat('/drive/MyDrive/PINNS/burgers_shock.mat')
mat_variables = [key for key in mat_data.keys() if not key.startswith('__')]
# fig = go.Figure(data=[go.Surface])
fig=go.Figure(go.Contour(z=mat_data['usol']))
fig.update_layout(xaxis_title='time',yaxis_title='x',xaxis_range=[0,100],yaxis_range=[0,256],
                  width=700,height=270,margin=dict(b=20,t=10))
fig.update_traces(zmin=mat_data['usol'].min(), zmax=mat_data['usol'].max())
fig.show()



In [3]:
class PINN:
  # initialize the class
  def __init__(self,X,u,layers,lb, ub):
    self.lb = lb # lower bound i think
    self.ub = ub # upper bound i think

    self.x = X[:,0:1] # first column of input data is displacement
    self.t = X[:,1:2] # 2nd column of input data is time
    self.u = u    # initial values of u for training

    self.layers = layers

    # initialize NNs
    self.weights, self.biases = self.initialize_NN(layers)

    # initialize parameters
    self.lambda_1= tf.Variable(0.0, dtype=tf.float32)
    self.lambda_2 = tf.Variable(-6.0, dtype=tf.float32)

    self.x_tf  = tf.convert_to_tensor(self.x, dtype = tf.float32)
    self.t_tf = tf.convert_to_tensor(self.t, dtype  = tf.float32)
    self.u_tf = tf.convert_to_tensor(self.u, dtype = tf.float32)

    self.u_pred = self.net_u(self.x_tf, self.t_tf)
    self.f_pred = self.net_f(self.x_tf, self.t_tf)

    self.loss = tf.reduce_mean(tf.square(self.u_tf-self.u_pred)) + \
                tf.reduce_mean(tf.square(self.f_pred))

    # Optimizer
    self.optimizer = tf.compact.v1.contrib.opt.ScipyOptimizerInterface(
        self.loss,
        method = 'L-BFGS-B',
        options = {'maxiter':50000,
                   'maxfun':50000,
                   'maxcor': 50,
                   'maxls':50,
                   'ftol': 1.0*np.finfo(float).eps}
    )
    self.optimizer_Adam = tf.keras.optimizers.Adam()
    self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)

    tf.compat.v1.disable_eager_execution()
    init = tf.compat.v1.global_variables_initializer()
    self.sess = tf.compat.v1.Session()
    self.sess.run(init)
  def initialize_NN(self, layers):
    weights = []
    biases = []
    num_layers  = len(layers)
    for l in range(0,num_layers-1):
      w = self.xavier_init(size=[layers[1], layers[l+1]])
      b = tf.Variable(tf.zeros([1,layers[l+1]],dtype=tf.float32),
                      dtype=tf.float32)
      weights.append(w)
      biases.append(b)
    return weights, biases
  def xavier_init(self,size):
    in_dim = size[0]
    out_dim = size[1]
    xavier_stddev = np.sqrt(2/(in_dim+out_dim))
    return tf.Variable(tf.truncated_normal([in_dim,out_dim],stddev=xavier_stddev),
                       dtype=tf.float32)

  def neural_net(self,X,weights, biases):
    num_layers = len(weights)+1
    H = 2.0*(X-self.lb)/(self.ub-self.lb)-1.0
    for l in range(0,num_layers-2):
      w=weights[l]
      b=weights[l]
      H = tf.tanh(tf.add(tf.matmul(H,w),b))
    w = weights[-1]
    b = biases[-1]
    Y = tf.add(tf.matmul(H,w),b)
    return Y

  def net_u(self,x,t):
      u = self.neural_net(tf.concat([x,t],1),self.weights,self.biases)
      return u

In [28]:
class ABC:

  def neural_net(self,X,weights, biases):
    self.lb=tf.reduce_min(X)
    self.ub=tf.reduce_max(X)
    num_layers = len(weights)+1
    H = 2.0*(X-self.lb)/(self.ub-self.lb)-1.0
    H=tf.cast(H,dtype=tf.float54)
    for l in range(0,num_layers-2):
      w=weights[l]
      b=weights[l]
      H = tf.tanh(tf.add(tf.matmul(H,w),b))
    w = weights[-1]
    b = biases[-1]
    Y = tf.add(tf.matmul(H,w),b)
    return Y
# s=ABC()
# s.neural_net(X,weights,biases)

In [80]:
##################### Self Modified
class PINN:
  # initialize the class
  # def __init__(self,X,u,layers,lb, ub):
  def __init__(self,X,layers):


    self.lb=tf.reduce_min(X)
    self.ub=tf.reduce_max(X)


    self.x = tf.reshape(X[:,0],(-1,1) )# first column of input data is displacement
    self.t = tf.reshape(X[:,1],(-1,1)) # 2nd column of input data is time
    # self.u = u    # initial values of u for training

    self.layers = layers

    # initialize NNs ,weights and biases
    self.weights, self.biases = self.initialize_NN(layers)
    self.u_pred = self.net_u()


  def initialize_NN(self, layers):
    weights = []
    biases = []
    num_layers = len(layers)
    for l in range(0, num_layers - 1):
        w = self.xavier_init(size=[layers[l], layers[l+1]])
        b = tf.Variable(tf.zeros([1, layers[l+1]], dtype=tf.float32), dtype=tf.float32)
        weights.append(w)
        biases.append(b)
    return weights, biases

  def xavier_init(self,size):   # xaiver weights and biases initialization
    in_dim = size[0]
    out_dim = size[1]
    xavier_stddev = np.sqrt(2/(in_dim+out_dim)).astype(np.float32)
    return tf.Variable(tf.random.truncated_normal([in_dim,out_dim],stddev=xavier_stddev),dtype=tf.float32)





  ##################### NN function
  def net_u(self):
    u = self.neural_net(tf.concat([self.x,self.t],1),self.weights,self.biases)
    return u

  def neural_net(self,X,weights, biases):
    num_layers = len(weights)+1
    H = 2.0*(X-self.lb)/(self.ub-self.lb)-1.0
    H=tf.cast(H,dtype=tf.float32)
    for l in range(0,num_layers-2):
      w=weights[l]
      b=biases[l]
      H=tf.cast(H,dtype=tf.float32)
      H = tf.tanh(tf.add(tf.matmul(H,w),b))
    w = weights[-1]
    b = biases[-1]
    Y = tf.add(tf.matmul(H,w),b)
    return Y
#################### Loss
    # self.loss = tf.reduce_mean(tf.square(self.u_tf-self.u_pred)) + \
    #             tf.reduce_mean(tf.square(self.f_pred))




In [82]:
np.random.seed(243)
tf.random.set_seed(4325)
x=mat_data['x'].astype(np.float32)
t=np.vstack((mat_data['t'],np.zeros((156,1))))
X=tf.concat([x,t],1)
layers = [2,3,2,1]
# layers = [2, 20, 20, 20, 20, 20, 20, 20, 20, 1]
pinn=PINN(X,layers)
biases=pinn.biases
weights=pinn.weights
pinn.net_u()


<tf.Tensor: shape=(256, 1), dtype=float32, numpy=
array([[-0.09027368],
       [-0.08671065],
       [-0.08311757],
       [-0.07949477],
       [-0.07584235],
       [-0.07216046],
       [-0.06844947],
       [-0.06470954],
       [-0.06094103],
       [-0.05714417],
       [-0.05331928],
       [-0.04946676],
       [-0.04558697],
       [-0.04168027],
       [-0.03774705],
       [-0.0337878 ],
       [-0.02980299],
       [-0.02579305],
       [-0.0217585 ],
       [-0.0176999 ],
       [-0.01361781],
       [-0.00951274],
       [-0.00538538],
       [-0.00123626],
       [ 0.00293392],
       [ 0.00712452],
       [ 0.01133481],
       [ 0.01556419],
       [ 0.01981182],
       [ 0.02407703],
       [ 0.02835901],
       [ 0.032657  ],
       [ 0.03697022],
       [ 0.04129784],
       [ 0.04563898],
       [ 0.04999284],
       [ 0.05435853],
       [ 0.0587352 ],
       [ 0.06312194],
       [ 0.06751782],
       [ 0.071922  ],
       [ 0.07633349],
       [ 0.08075139],
    

In [77]:
X.dtype

tf.float32

In [56]:
x.d


dtype('float64')

In [40]:
# np.concatenate((mat_data['x'],mat_data['t']),1)
# tf.concat([mat_data['x'],mat_data['t']],1)
x=mat_data['x']
t=np.vstack((mat_data['t'],np.zeros((156,1),dtype=np.float32)))
X=tf.concat([x,t],1)
X

<tf.Tensor: shape=(256, 2), dtype=float64, numpy=
array([[-1.        ,  0.        ],
       [-0.99215686,  0.01      ],
       [-0.98431373,  0.02      ],
       [-0.97647059,  0.03      ],
       [-0.96862745,  0.04      ],
       [-0.96078431,  0.05      ],
       [-0.95294118,  0.06      ],
       [-0.94509804,  0.07      ],
       [-0.9372549 ,  0.08      ],
       [-0.92941176,  0.09      ],
       [-0.92156863,  0.1       ],
       [-0.91372549,  0.11      ],
       [-0.90588235,  0.12      ],
       [-0.89803922,  0.13      ],
       [-0.89019608,  0.14      ],
       [-0.88235294,  0.15      ],
       [-0.8745098 ,  0.16      ],
       [-0.86666667,  0.17      ],
       [-0.85882353,  0.18      ],
       [-0.85098039,  0.19      ],
       [-0.84313725,  0.2       ],
       [-0.83529412,  0.21      ],
       [-0.82745098,  0.22      ],
       [-0.81960784,  0.23      ],
       [-0.81176471,  0.24      ],
       [-0.80392157,  0.25      ],
       [-0.79607843,  0.26      ],
     

In [67]:
np.random.seed(243)
H=tf.constant(np.random.randint(0,20,(10,2)),dtype=tf.float32)
# w=tf.constant(np.random.rand(10,1),dtype=tf.float32)
H1=tf.matmul(H,w[0])
H2=tf.matmul(H1,w[1])
tf.matmul(H2,w[2])

<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[-1.5841147 ],
       [-0.5887659 ],
       [ 0.9175893 ],
       [ 0.75095856],
       [ 0.18811178],
       [-0.5991336 ],
       [-0.35918236],
       [ 0.6309821 ],
       [ 1.4804354 ],
       [ 1.7566746 ]], dtype=float32)>